# Trash

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
import cv2

path_to_dataset = "../dataset/dataset512x512/RECHERCHE-015.npy"
path_to_labels = "../dataset/dataset512x512/RECHERCHE-015_label.npy"

# Extrapolate Glomeruli

In [2]:
dataset = np.load(path_to_dataset)
labels = np.load(path_to_labels)

In [3]:
# Identification of images and labels with glomerulus
glomeruli = np.array([dataset[i] for i in range(len(dataset)) if (1 in labels[i])])
glomeruli_labels = np.array([labels[i] for i in range(len(dataset)) if (1 in labels[i])])

# Identification of images and labels without glomerulus
notGlomeruli = np.array([dataset[i] for i in range(len(dataset)) if not(1 in labels[i])])
notGlomeruli_labels = np.array([labels[i] for i in range(len(dataset)) if not(1 in labels[i])])

In [1]:
def glomeruli_crop(glomeruli, glomeruli_labels):
    result = []
    original_images = []
    original_labels = []
    result_labels = []
    
    for j in range(0, len(glomeruli)):
        num_labels, labels = cv2.connectedComponents(
            glomeruli_labels[j], connectivity=8)
        for i in range(1, num_labels):
            image = glomeruli[j]
            original_image = image.copy()  # Save the original image
            original_images.append(original_image)
            original_label = labels.copy()  # Save the original label
            original_labels.append(original_label)

            # Find row and column indices of pixels with value 1
            yrow, xcol = np.where(labels == i)

            # Calculate minimum and maximum row indices
            yrowmin = np.min(yrow)
            yrowmax = np.max(yrow)

            # Calculate minimum and maximum column indices
            xcolmin = np.min(xcol)
            xcolmax = np.max(xcol)

            # Calculate the size of the side of the square as the max between the height and the width
            side = max(yrowmax - yrowmin + 1, xcolmax - xcolmin + 1)
            if (side >= 20):
                # Adjust the bounding box to be square
                yrowmin, yrowmax, xcolmin, xcolmax = adjust_bounding_box(yrowmin, yrowmax, xcolmin, xcolmax, side, image.shape)

                # Extract the square image from the original image
                cropped_image = image[yrowmin:yrowmax, xcolmin:xcolmax]

                # Calculate the proportion of glomerulus pixels in the cropped image
                cropped_label = labels[yrowmin:yrowmax, xcolmin:xcolmax] == i
                glomerulus_proportion = np.mean(cropped_label)
                
                # Only keep the image if the glomerulus proportion is above the threshold
                if glomerulus_proportion >= 0.4:
                    # Resize image
                    resized_image = cv2.resize(
                        cropped_image, (200, 200), interpolation=cv2.INTER_AREA)
                    result.append(resized_image)
                    result_labels.append(i)  # Save the label of the cropped image

    return np.array(result), np.array(result_labels), np.array(original_images), np.array(original_labels)


In [5]:
result, result_label, ori, ori_l = glomeruli_crop(glomeruli, glomeruli_labels)


In questo codice, dopo aver calcolato le dimensioni del lato del quadrato, regoliamo le coordinate minime e massime del bounding box per ottenere una forma quadrata, assicurandoci che il bounding box non esca dai bordi dell'immagine. Infine, estrapoliamo l'immagine quadrata dall'immagine originale.

In [6]:
result_label.shape

(84, 200, 200)

In [ ]:
# Save each image in a different file 
for i in range(0, 20):
    plt_image_labels(result[i], result_label[i], ori[i], ori_l[i])
    #plt_image(result[i])
    #cv2.imwrite(f'image{i}.jpg', result[i])


In [13]:
from matplotlib import pyplot as plt
import cv2
def plt_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [31]:
def plt_image(image, labels):

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Sovrapposizione punti con labels=1
    overlay = np.zeros_like(image)
    overlay[np.where(labels == 1)] = [128, 128, 0]  # Giallo (BGR)
    #overlay[np.where(labels == 0)] = [255, 255, 255]  # Giallo (BGR)
    
    alpha = 0.8  # Livello di trasparenza
    overlay = cv2.addWeighted(image, 1, overlay, alpha, 0)

    plt.imshow(overlay)  # Sovrapposizione con trasparenza
    plt.show()

In [8]:
def plt_image_labels(image, labes, im, lab):
    fig, axs = plt.subplots(2, 2)
    # Primo subplot: labels
    axs[0][0].imshow(labes)
    axs[0][0].axis('off')
    # Secondo subplot: immagine
    axs[0][1].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[0][1].axis('off')
    # Mostra i subplot affiancati
    axs[1][0].imshow(lab)
    axs[1][0].axis('off')
    # Secondo subplot: immagine
    axs[1][1].imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    axs[1][1].axis('off')
    plt.show()